ORtrack tracker 

Import clause:

a) Import modules

In [1]:
import torch
import cv2
import sys
import importlib
import time
import numpy as np
import os

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

ModuleNotFoundError: No module named 'pycuda'

In [2]:
!pip install pycuda

  Using cached pycuda-2025.1.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pytools-2025.1.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached siphash24-1.7.tar.gz (19 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      + meson setup C:\Users\varaksa_yua\AppData\Local\Temp\pip-install-03cxie_g\siphash24_80118e84a9f741e6ad50e14d7b45448e C:\Users\varaksa_yua\AppData\Local\Temp\pip-install-03cxie_g\siphash24_80118e84a9f741e6ad50e14d7b45448e\.mesonpy-fqr19mfj -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\varaksa_yua\AppData\Local\Temp\pip-install-03cxie_g\siphash24_80118e84a9f741e6ad50e14d7b45448e\.mesonpy-fqr19mfj\meson-python-native-file.ini
      The Meson build system
      Version: 1.8.1
      Source dir: C:\Users\varaksa_yua\AppData\Local\Temp\pip-install-03cxie_g\siphash24_80118e84a9f741e6ad50e14d7b45448e
      Build dir: C:\Users\varaksa_yua\AppData\Local\Temp\pip-install-03cxie_g\siphash24_80118e84a9f741e6ad50e14d7b45448e\.mesonpy-fqr19mfj
      Build type: native build
      Activating VS 17.14.2
      Project

b) Import classes and functions from additional files

In [2]:
from lib.models.ortrack import build_ortrack
from lib.test.tracker.basetracker import BaseTracker

from lib.test.utils.hann import hann2d
from lib.train.data.processing_utils import sample_target

from lib.test.tracker.data_utils import Preprocessor
from lib.utils.box_ops import clip_box
from lib.utils.ce_utils import generate_mask_cond

c:\Users\User\anaconda3\envs\ortrack\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\anaconda3\envs\ortrack\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\User\Documents\MLearning\_ORTrack_v8_trt\lib\models\ortrack\vision_transformer.py:1798: UserWarning: Overwriting vit_tiny_patch16_224 in registry with lib.models.ortrack.vision_transformer.vit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def vit_tiny_patch16_224(pretrained: bool = False, **kwargs) -> VisionTransformer:
c:\Users\User\Documents\MLearnin

Definition clause

In [3]:
# Class for wrapping loaded TensorRT model
KEYS = ['pred_boxes', 'score_map', 'size_map', 'offset_map', 'backbone_feat'] # new

class TrtModelWrapper:
    def __init__(self, trt_model_path):
        """Загрузка TensorRT модели"""
        with open(trt_model_path, "rb") as f:
            engine_data = f.read()
        self.runtime = trt.Runtime(trt.Logger())
        self.engine = self.runtime.deserialize_cuda_engine(engine_data)
        self.context = self.engine.create_execution_context()
        print(f"Context profile #: {self.context.engine.num_optimization_profiles}")

        # Получаем имена тензоров                
        self.tensor_names = [self.engine.get_tensor_name(i) for i in range(self.engine.num_io_tensors)]
        self.input_tensors = self.tensor_names[0:2]  # Исключаем выходные тензоры
        self.input_shapes = [self.context.get_tensor_shape(name) for name in self.input_tensors]
        self.output_tensors = self.tensor_names[2:]  # Исключаем входные тензоры
        self.output_shapes = [self.context.get_tensor_shape(name) for name in self.output_tensors]
        
        # переменные под GPU память
        self.d_inputs = {}
        self.d_outputs = {}

        self.allocate_memory()


    def allocate_memory(self):
        """Выделение правильного объема памяти в GPU"""

        # Выделяем память под входные тензоры
        for name, shape in zip(self.input_tensors, self.input_shapes):
            size = int(np.prod(shape) * np.dtype(np.float32).itemsize)
            self.d_inputs[name] = cuda.mem_alloc(size)

        # Выделяем память под выходные тензоры
        for name, shape in zip(self.output_tensors, self.output_shapes):
            size = int(np.prod(shape) * np.dtype(np.float32).itemsize)
            self.d_outputs[name] = cuda.mem_alloc(size)


    def run_inference(self, z_numpy, x_numpy):
        """Запуск инференса для текущего кадра"""
        
        z_numpy = z_numpy.astype(np.float32).copy(order='C')
        x_numpy = x_numpy.astype(np.float32).copy(order='C')

        cuda.memcpy_htod(self.d_inputs["z"], z_numpy)
        cuda.memcpy_htod(self.d_inputs["x"], x_numpy)

        # Создаем список всех входных и выходных буферов
        bindings = [int(self.d_inputs["z"]), int(self.d_inputs["x"])] + [int(self.d_outputs[name]) for name in self.output_tensors]
        
        # Запускаем инференс
        self.context.execute_v2(bindings)

    def copy_results_to_cpu(self):
        """Извлекаем результаты инференса из GPU"""
        cpu_outputs = {}
        i = 0
        for name in self.output_tensors:
            cpu_outputs[KEYS[i]] = np.empty(self.output_shapes[self.output_tensors.index(name)], dtype=np.float32)
            cuda.memcpy_dtoh(cpu_outputs[KEYS[i]], self.d_outputs[name])
            i +=1
        # print(f"type(cpu_outputs): {type(cpu_outputs)}") 
        return cpu_outputs

In [5]:
# KEYS = ['pred_boxes', 'score_map', 'size_map', 'offset_map', 'backbone_feat'] # new
class ORTrack_new(BaseTracker):
    def __init__(self, params, trt_model_path):
        super(ORTrack_new, self).__init__(params)
        network = build_ortrack(params.cfg, training=False)
        network.load_state_dict(torch.load(self.params.checkpoint, map_location='cpu', weights_only = False)['net'], strict=True)
        self.network = network.cuda() if torch.cuda.is_available() else network.cpu()
        self.network.eval()
        
        self.cfg = params.cfg
        self.preprocessor = Preprocessor()
        self.state = None

        self.feat_sz = self.cfg.TEST.SEARCH_SIZE // self.cfg.MODEL.BACKBONE.STRIDE
        self.feat_template_sz = self.cfg.TEST.TEMPLATE_SIZE // self.cfg.MODEL.BACKBONE.STRIDE
        # motion constrain
        try:
            self.output_window = hann2d(torch.tensor([self.feat_sz, self.feat_sz]).long(), centered=True).cuda()
        except:
            self.output_window = hann2d(torch.tensor([self.feat_sz, self.feat_sz]).long(), centered=True).cpu()

        # new thing (here)
        self.frame_id = 0 

        # for save boxes from all queries
        self.save_all_boxes = params.save_all_boxes
        self.z_dict = {}
        self.is_distill = params.cfg.MODEL['IS_DISTILL']

        # new thing!!!
        self.loaded_model = TrtModelWrapper(trt_model_path)                  

    def initialize(self, image, info: dict):
        # forward the template once
        z_patch_arr, resize_factor, z_amask_arr = sample_target(image, info['init_bbox'], self.params.template_factor,
                                                    output_sz=self.params.template_size)
        # cv2.imshow("ffdd",z_patch_arr)
        self.z_patch_arr = z_patch_arr
        template = self.preprocessor.process(z_patch_arr, z_amask_arr)
        with torch.no_grad():
            self.z_dict = template

        self.box_mask_z = None
        if self.cfg.MODEL.BACKBONE.CE_LOC:
            template_bbox = self.transform_bbox_to_crop(info['init_bbox'], resize_factor,
                                                        template.tensors.device).squeeze(1)
            self.box_mask_z = generate_mask_cond(self.cfg, 1, template.tensors.device, template_bbox)

        # save states
        self.state = info['init_bbox']
        self.frame_id = 0
        if self.save_all_boxes:
            '''save all predicted boxes'''
            all_boxes_save = info['init_bbox'] * self.cfg.MODEL.NUM_OBJECT_QUERIES
            return {"all_boxes": all_boxes_save}


    def track(self, image, info: dict = None):
        H, W, _ = image.shape
        self.frame_id += 1
        x_patch_arr, resize_factor, x_amask_arr = sample_target(image, self.state, self.params.search_factor,
                                                                output_sz=self.params.search_size)  # (x1, y1, w, h)
        # cv2.imshow("ff",x_patch_arr)
        search = self.preprocessor.process(x_patch_arr, x_amask_arr)

        with torch.no_grad():
            x_dict = search
            # merge the template and the search
            # run the transformer
            # old:
            # out_dict = self.network.forward(
            #     template=self.z_dict.tensors, search=x_dict.tensors, is_distill=self.is_distill)
            # new:
            # z_numpy = self.z_dict.tensors.cpu().numpy().copy()
            # x_numpy = x_dict.tensors.cpu().numpy().copy()      
            z_numpy = self.z_dict.tensors.cpu().numpy().astype(np.float32).copy()
            x_numpy = x_dict.tensors.cpu().numpy().astype(np.float32).copy()    
            self.loaded_model.run_inference(z_numpy, x_numpy)
            output = self.loaded_model.copy_results_to_cpu()
            out_dict = {}
            for k in output.keys():
                out_dict[k] = torch.from_numpy(output[k]).to('cuda')


        # add hann windows
        pred_score_map = out_dict['score_map']
        # print(f"pred_score_map type: {type(pred_score_map)}")
        # print(f"self.output_window type: {type(self.output_window)}")
        response = self.output_window * pred_score_map
        pred_boxes = self.network.box_head.cal_bbox(response, out_dict['size_map'], out_dict['offset_map'])
        pred_boxes = pred_boxes.view(-1, 4)
        
        # Baseline: Take the mean of all pred boxes as the final result
        pred_box = (pred_boxes.mean(dim=0).cpu().numpy() * self.params.search_size / resize_factor).tolist()  # (cx, cy, w, h) [0,1]
        # get the final box result
        self.state = clip_box(self.map_box_back(pred_box, resize_factor), H, W, margin=10)
        # old:
        # return {"target_bbox": self.state} 
        # new:
        confidence = response.max().item() 
        return {"target_bbox": self.state, "confidence": confidence}  

    def map_box_back(self, pred_box: list, resize_factor: float):
        cx_prev, cy_prev = self.state[0] + 0.5 * self.state[2], self.state[1] + 0.5 * self.state[3]
        cx, cy, w, h = pred_box
        half_side = 0.5 * self.params.search_size / resize_factor
        cx_real = cx + (cx_prev - half_side)
        cy_real = cy + (cy_prev - half_side)
        return [cx_real - 0.5 * w, cy_real - 0.5 * h, w, h]

    def map_box_back_batch(self, pred_box: torch.Tensor, resize_factor: float):
        cx_prev, cy_prev = self.state[0] + 0.5 * self.state[2], self.state[1] + 0.5 * self.state[3]
        cx, cy, w, h = pred_box.unbind(-1) # (N,4) --> (N,)
        half_side = 0.5 * self.params.search_size / resize_factor
        cx_real = cx + (cx_prev - half_side)
        cy_real = cy + (cy_prev - half_side)
        return torch.stack([cx_real - 0.5 * w, cy_real - 0.5 * h, w, h], dim=-1)


def get_tracker_class():
    return ORTrack_new

Loading clause

In [6]:
param_module = importlib.import_module('lib.test.parameter.ortrack')
yaml_name = "deit_tiny_patch16_224.yaml"

model_name1 = "c:/Users/User/Documents/MLearning/_ORTrack_v0-initial/Model/deit_tiny_patch16_224/ORTrack_ep0300.pth.tar" # at home
model_name2 = "d:/CurrentWork/ML/_ORTrack_old/ORTrack_ep0300.pth.tar" # at work
# checking which file exists on the current PC:
if os.path.isfile(model_name1):
    model_name = model_name1
elif os.path.isfile(model_name2):
    model_name = model_name2

params = param_module.parameters(yaml_name, model_name)

test config:  {'MODEL': {'PRETRAIN_FILE': '', 'IS_DISTILL': False, 'EXTRA_MERGER': False, 'RETURN_INTER': False, 'RETURN_STAGES': [], 'BACKBONE': {'TYPE': 'deit_tiny_patch16_224', 'STRIDE': 16, 'MID_PE': False, 'SEP_SEG': False, 'CAT_MODE': 'direct', 'MERGE_LAYER': 0, 'ADD_CLS_TOKEN': False, 'CLS_TOKEN_USE_MODE': 'ignore', 'CE_LOC': [], 'CE_KEEP_RATIO': [], 'CE_TEMPLATE_RANGE': 'ALL'}, 'HEAD': {'TYPE': 'CENTER', 'NUM_CHANNELS': 256}}, 'TRAIN': {'LR': 0.0004, 'WEIGHT_DECAY': 0.0001, 'EPOCH': 300, 'LR_DROP_EPOCH': 240, 'BATCH_SIZE': 32, 'NUM_WORKER': 10, 'OPTIMIZER': 'ADAMW', 'BACKBONE_MULTIPLIER': 0.1, 'GIOU_WEIGHT': 2.0, 'L1_WEIGHT': 5.0, 'FREEZE_LAYERS': [0], 'PRINT_INTERVAL': 50, 'VAL_EPOCH_INTERVAL': 20, 'GRAD_CLIP_NORM': 0.1, 'AMP': False, 'CE_START_EPOCH': 20, 'CE_WARM_EPOCH': 80, 'DROP_PATH_RATE': 0.1, 'SCHEDULER': {'TYPE': 'step', 'DECAY_RATE': 0.1}}, 'DATA': {'SAMPLER_MODE': 'causal', 'MEAN': [0.485, 0.456, 0.406], 'STD': [0.229, 0.224, 0.225], 'MAX_SAMPLE_INTERVAL': 200, 'TRAI

In [7]:
trt_model_path1 = "c:/Users/User/Documents/MLearning/_ORTrack_v8_trt/ORTrack.trt"      # at home
trt_model_path2 = "d:/CurrentWork/ML/_ORTrack_v8_trt/ORTrack.trt"                      # at work

# checking which file exists on the current PC:
if os.path.isfile(trt_model_path1):
    trt_model_path = trt_model_path1
elif os.path.isfile(trt_model_path2):
    trt_model_path = trt_model_path2

trt_model_path

'c:/Users/User/Documents/MLearning/_ORTrack_v8_trt/ORTrack.trt'

Loading TensorRT model, initializing the tracker

In [ ]:
trt_model = TrtModelWrapper(trt_model_path)
tracker = ORTrack_new(params, trt_model_path)

Context profile #: 1


Execution clause

In [20]:
# Video file to process
path1 = "c:/Users/User/Documents/MLearning/_VideosForTracking/" # at home
path2 = "d:/CurrentWork/ML/_VideosForTracking/" # at work
if os.path.isdir(path1):
    path = path1
elif os.path.isdir(path2):
    path = path2    

file = "0516.mp4"               # blue target from drone
# file = 'PKP_TV_UPZx2_3.mp4'   # tank narrow FOV
# file = "PNK_TV_SHPZ_1.mp4"    # tank wide FOV
# file = "cat.mp4"              # cat
# file = "08_53_38_CAM.mp4"       # piazza
# file = u"13-перекресток_крыша_Пеленга_туман_ТВ.mp4" # "u" means unicode string
# file = u"15-повозка_лошадь_человек_ТВ.mp4"
# file = u"22-грузовик_ТВ.mp4"         # truck in a field
# file = u"30-Наряд_ТВ.mp4"            # couple of soldiers 
# file = u"3-10_39_36_ТВ_авто.mp4"   # car on a country road near border, moving away
# file = u"7-09_57_49_ТВ_авто.mp4"   # car on a country road near border, approaching
# file = "Road_Colored_14.20.21.mp4"  # cross-road Makajonka

In [21]:
video = cv2.VideoCapture(path + file)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=video.get(cv2.CAP_PROP_FPS)

video_writer = cv2.VideoWriter(file.split('.')[0]+"__"+".avi", fourcc, fps, (width, height))

ok, image = video.read()
if not video.isOpened():
    print("Could not open video")
    sys.exit()
    
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # not needed since we read, show and save pictures with cv2 which uses BGR throughout

x, y, w, h = cv2.selectROI(image, fromCenter=False)
init_state = [x, y, w, h]
def _build_init_info(box):
            return {'init_bbox': box}
tracker.initialize(image, _build_init_info(init_state))
counter = 0
frame_intervals = [] # frame intervals for calsulation processing FSP
# fontScale
fontScale = 1
font = cv2.FONT_HERSHEY_SIMPLEX
# Blue color in BGR
color = (0, 0, 200)
# Line thickness of 2 px
thickness = 2 
while True:
            t1 = time.perf_counter() # precision timer start
            ok, image = video.read()
            if not ok:
                print("Can't read frame")
                break
            
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # not needed since we read, show and save pictures with cv2 which uses BGR throughout
            start = time.time() 
            # print(f"Images size fed to the tracker: {image.shape}")
            out  = tracker.track(image)
            state = [int(s) for s in out['target_bbox']]
            best_score=out["confidence"]
            end_time = (time.time() - start)            
            
            org = (50, 50)

            # Using cv2.putText() method
            image = cv2.putText(image, str(best_score), org, font, 
                            fontScale, color, thickness, cv2.LINE_AA)
            image = cv2.putText(image, str(end_time), (50,100), font, 
                            fontScale, color, thickness, cv2.LINE_AA)

            x, y, w, h = [int(x) for x in state]

            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

            cv2.imshow("tracking", image)
            video_writer.write(image)
            t2 = time.perf_counter() # timer stop
            frame_intervals.append(t2 - t1)
            
            k = cv2.waitKey(1)  
            if k == 32:  # SPACE
                  ok, image = video.read()
                  if not ok:
                         print("Can't read frame for ROI selection")
                         continue  # Пропускаем этот кадр
                  x, y, w, h = cv2.selectROI(image, fromCenter=False)
                  init_state = [x, y, w, h]
                  tracker.initialize(image, _build_init_info(init_state))

            if k == 27:  # ESC
                break
        
                
cv2.destroyAllWindows()
video.release()
video_writer.release()

Performance evaluation:

In [22]:
print(f'Video size: {width} * {height}\n')

# frame_intervals = [i for i in frame_intervals if i > 0] # removing zero time values
FPS = [1/i for i in frame_intervals]
print(f'Min FPS: {min(FPS):.2f}')
print(f'Max FPS: {max(FPS):.2f}')
FPS_ave = np.mean(FPS)
print(f'Average FPS of video tracking: {FPS_ave:.2f}')
MRSE = np.std(FPS)
print(f'Standard deviation of FPS video tracking: {MRSE:.2f}')

Video size: 1280 * 720

Min FPS: 16.15
Max FPS: 98.92
Average FPS of video tracking: 81.69
Standard deviation of FPS video tracking: 8.49
